In [ ]:
import csv
import json
import logging
import pandas as pd
import requests
from github import Github
from github.GithubException import GithubException

In [ ]:
GITHUB_TOKEN = ""
g = Github(GITHUB_TOKEN)

In [ ]:
df = pd.read_excel("dropped_duplicates_pr.xlsx")
df.head(3)

In [ ]:
len(df)

In [ ]:
df_first = df[df['round_id'] == 0]
len(df_first)

In [ ]:
df_first.to_excel("df_first_pr.xlsx", engine = "openpyxl", index=False)

In [ ]:
#manual removing non-english and 404 items. 

In [ ]:
df_first = pd.read_excel("df_first_pr_english_only.xlsx")
len(df_first)

In [ ]:
df_first_no_404 = pd.read_excel("df_first_pr_english_only_non_404.xlsx")
len(df_first_no_404)

In [ ]:
pr_num_participants = []

In [ ]:
def get_pr_participants(repo_name, pull_number):
    #pr_num_participants = []
    try:
        repo = g.get_repo(f"{repo_name}")
        #print("here")
        # Get the pull request
        pull_request = repo.get_pull(pull_number)
        #print("here")
        # Get comments from the PR
        comments = pull_request.get_issue_comments()
        #print("here")
        # Get reviews from the PR
        reviews = pull_request.get_reviews()
        #print("here")
        # Extract unique user logins from comments and reviews
        user_logins = set()
        for comment in comments:
            user_logins.add(comment.user.login)
        #for review in reviews:
            #user_logins.add(review.user.login)
        # The number of unique participants
        number_of_participants = len(user_logins)
        #print(len(user_logins))
    except GithubException as e:
        if e.status == 404:
            # Handle the 404 error by appending a placeholder
            number_of_participants = -1
        elif e.status == 403:
            number_of_participants = -1
        else:
            number_of_participants = -1

    pr_num_participants.append(number_of_participants)


In [ ]:
for _, row in df_first_no_404.iterrows():
    repo_name = row['RepoName']
    pull_number = row['PullNumber']
    get_pr_participants(repo_name, pull_number)

In [ ]:
len(pr_num_participants)

In [ ]:
pr_num_participants[109]

In [ ]:
pr_df =pd.DataFrame(pr_num_participants, columns=['num_participants'])
pr_df

In [ ]:
df_first_no_404["num_participants"] = pr_num_participants
df_first_no_404

In [ ]:
df_first["MentionedProperty"].value_counts()

In [ ]:
pr_df.value_counts()

In [ ]:
average = pr_df.mean()
print(average)

In [ ]:
std_dev = pr_df.std()
print(std_dev)

In [ ]:
df_first_no_404.to_excel("rq1_pr_english_only_no_404.xlsx", engine='openpyxl', index=False)